In [291]:
import requests
import time
import tqdm
import pandas as pd
# https://www.hindawi.com/oai-pmh/

### Gather data

In [104]:

delta_data = 200
t_wait = 0.5 # waiting time
data_total = []

# Init
url_bielfeld = 'http://pub.uni-bielefeld.de/oai?verb=ListRecords&metadataPrefix=oai_datacite'
x = requests.get(url_bielfeld)
o = xmltodict.parse(x.content)

data = o['OAI-PMH']['ListRecords']['record'] 
r_token = o['OAI-PMH']['ListRecords']['resumptionToken']['#text']
[data_total.append(d) for d in data]

while True:
    url_bielfeld = 'https://pub.uni-bielefeld.de/oai?verb=ListRecords&resumptionToken='+r_token
    x = requests.get(url_bielfeld)
    o = xmltodict.parse(x.content)
    data = o['OAI-PMH']['ListRecords']['record'] 
    if not o['OAI-PMH']['ListRecords']['resumptionToken'].get('#text', False):
        print('data loaded')
        break
    r_token = o['OAI-PMH']['ListRecords']['resumptionToken']['#text']
    [data_total.append(d) for d in data]
    print(len(data_total))
    time.sleep(t_wait)


400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
10000
10200
10400
10600
10800
11000
11200
11400
11600
11800
12000
12200
12400
12600
12800
13000
13200
13400
13600
13800
14000
14200
14400
14600
14800
15000
15200
15400
15600
15800
16000
16200
16400
16600
16800
17000
17200
17400
17600
17800
18000
18200
18400
18600
18800
19000
19200
19400
19600
19800
20000
20200
20400
20600
20800
21000
21200
21400
21600
21800
22000
22200
22400
22600
22800
23000
23200
23400
23600
23800
24000
24200
24400
24600
24800
25000
25200
25400
25600
25800
26000
26200
26400
26600
26800
27000
27200
27400
27600
27800
28000
28200
28400
28600
28800
29000
29200
29400
29600
29800
30000
30200
30400
30600
30800
31000
31200
31400
31600
31800
32000
32200
32400
32600
32800
33000
33200
33400
33600
33800
34000
34200
34400
34600
34800
35000
35200
3

### Functions//getter

In [293]:

def get_identifier(d):
    return d['header']['identifier']

def get_timestamps(d):
    return d['header']['datestamp']

def get_title(d):
    if type(d['metadata']['resource']['titles']['title'])!=list:
        return d['metadata']['resource']['titles']['title'].lower()
    else:
        return d['metadata']['resource']['titles']['title'][1]['#text'].lower()

def get_subjects(d):
    if d['metadata']['resource'].get('subjects', False):
        return [s.lower() for s in d['metadata']['resource']['subjects']['subject'] if type(s)==str]  
    else: 
        return []
    
def get_orci_authors(d):
    if type(d['metadata']['resource']['creators']['creator']) == list:
        return [{'orcid': c['nameIdentifier']['#text'].lower()} for c in d['metadata']['resource']['creators']['creator'] if c.get('nameIdentifier', False)]
    else:
        return [{'orcid': d['metadata']['resource']['creators']['creator']['nameIdentifier']['#text'].lower() if d['metadata']['resource']['creators']['creator'].get('nameIdentifier', False) else []}] 
                 
def get_authors(d):
    if type(d['metadata']['resource']['creators']['creator']) == list:
        return [{'givenName':c['givenName'].lower(), 'familyName': c['familyName'].lower()} for c in
                d['metadata']['resource']['creators']['creator']]
    else:
        return [{'givenName': d['metadata']['resource']['creators']['creator']['givenName'],
                 'familyName': d['metadata']['resource']['creators']['creator']['familyName']}]

def get_data(d, label):
    try: 
        if label=='identifier':
            return get_identifier(d)
        elif label=='timestamp':
            return get_timestamps(d)
        elif label=='authors':
            return get_authors(d)
        elif label=='orci_authors':
            return get_orci_authors(d)
        elif label=='title':
            return get_title(d)
        elif label=='subject_list': 
            return get_subjects(d)
    except:
        return 'error::'+label

    
def get_data_dict(d, ks_data):    
    return dict([(k,get_data(d,k)) for k in ks_data])


### Process data

In [294]:

ks_data = ['identifier','timestamp','authors','orci_authors','title','subject_list']

processed_data = []
for i in tqdm.tqdm(range(len(data_total))):
    #print(i)
    d = data_total[i]
    processed_data.append(get_data_dict(d, ks_data))


100%|██████████| 74200/74200 [00:00<00:00, 76444.00it/s]


### Build Dataframe

In [296]:
df = pd.DataFrame(dict([(k,[d[k] for d in processed_data]) for k in k_data]))
df

,identifier,timestamp,authors,orci_authors,title,subject_list
0,oai:pub.uni-bielefeld.de:1648257,2020-10-27T12:41:58Z,"[{'givenName': 'achim', 'familyName': 'bachem'...",[],five variations on a theme by gyula farkas,[]
1,oai:pub.uni-bielefeld.de:1648277,2018-07-24T12:58:19Z,"[{'givenName': 'Rüdiger', 'familyName': 'Bittn...",[{'orcid': []}],is it reasonable to regret things one did?,[]
2,oai:pub.uni-bielefeld.de:1648283,2018-07-24T12:58:53Z,"[{'givenName': 'k', 'familyName': 'knuttel'}, ...",[{'orcid': '0000-0003-0117-4021'}],(model) studies on vanadate-dependent bromo io...,"[v-51 nuclear, electron paramagnetic resonance..."
3,oai:pub.uni-bielefeld.de:1648289,2020-10-27T09:37:20Z,"[{'givenName': 'andreas', 'familyName': 'dress...",[],convex tree realizations of partitions,[]
4,oai:pub.uni-bielefeld.de:1648292,2018-07-24T12:59:12Z,"[{'givenName': 'farhad', 'familyName': 'faisal...",[],prediction of a new sequence of peaks in above...,[]
...,...,...,...,...,...,...
74195,oai:pub.uni-bielefeld.de:2954718,2021-05-21T09:34:34Z,"[{'givenName': 'matthias', 'familyName': 'preu...",[],bêtes studies: flaubert’s and balzac’s lessons...,[]
74196,oai:pub.uni-bielefeld.de:2954735,2021-05-21T09:34:34Z,"[{'givenName': 'christoph', 'familyName': 'kie...",[{'orcid': '0000-0002-9166-400x'}],treatment effects on count outcomes with non‐n...,[]
74197,oai:pub.uni-bielefeld.de:2954851,2021-05-21T09:34:34Z,"[{'givenName': 'laziz', 'familyName': 'turaev'...",[],universal access to xpert mtb/rif testing for ...,[]
74198,oai:pub.uni-bielefeld.de:2954701,2021-05-21T09:34:35Z,"[{'givenName': 'manuel', 'familyName': 'batram...",[{'orcid': '0000-0003-0350-360x'}],burden of herpes zoster in adult patients with...,[]
